# AWS Suspicious SAML Activity

Identifies when suspicious SAML activity has occurred in AWS. An adversary could gain backdoor access via SAML.

References:
* https://docs.aws.amazon.com/IAM/latest/APIReference/API_UpdateSAMLProvider.html
* https://docs.aws.amazon.com/STS/latest/APIReference/API_AssumeRoleWithSAML.html

False Positives:
* Automated processes that uses Terraform may lead to false positives.
* SAML Provider could be updated by a system administrator. Verify whether the user identity, user agent, and/or hostname should be making changes in your environment.
* SAML Provider being updated from unfamiliar users should be investigated. If known behavior is causing false positives, it can be exempted from the rule.

## Detection Query
Execute the following query to find alerts related to this rule.

In [ ]:
%ingest.source_type="aws:cloudtrail"
(
eventSource=sts.amazonaws.com
and
eventName=AssumeRoleWithSAML
)
or
(
eventSource=iam.amazonaws.com
and
eventName=UpdateSAMLProvider
)
| stats
min(@scnr.datetime) as firstTime,
max(@scnr.datetime) as lastTime,
count() as eventCount
by
userIdentity.arn,
eventSource,
eventName,
awsRegion

## Investigation Steps
1. **Identify the Principal**: Who performed the action?
2. **Review Context**: What else did this principal do?
3. **Check Permissions**: Did the principal have legitimate reasons for this action?
4. **Verify Resource**: Is the resource critical or sensitive?

In [ ]:
%ingest.source_type:aws:cloudtrail
userIdentity.arn: <PRINCIPAL_ARN>
| count(eventName) by eventName